In [1]:
# connect mysql
import pymysql
conn = pymysql.connect(host = 'localhost', user='dongyeon0317', password='1Q2w3e4r!@', db='crime_prediction', charset = 'utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)

In [28]:
# select from Dataset - Choose scenario

# Season
num = int(input('1.Spring 2.Summer 3.Fall 4.Winter '))
if num == 1: season = 'Spring'
elif num == 2: season = "Summer" 
elif num == 3: season = "Fall" 
else: season = "Winter"

# Time    
num = int(input("1.Morning 2.Afternoon 3.Evening 4.Midnight "))
if num == 1: time = "Morning"  
elif num == 2: time = "Afternoon" 
elif num == 3: time = "Evening" 
else: time = "Midnight"
    
# Weather    
num = int(input("1.눈/비 2.맑음 3.흐림 "))
if num == 1: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall > 0) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall > 0);' 
elif num == 2: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud < 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud < 5);' 
else: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud >= 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud >= 5);' 


dataset_sql = 'SELECT grid, CCTV_num, police_value, resident, floating FROM Dataset ' + where
grid_sql = 'SELECT grid, CCTV_num, police_value, resident, floating FROM Grid WHERE time = \''+ time + '\''
curs.execute(dataset_sql)
dataset_result = curs.fetchall()
curs.execute(grid_sql)
grid_result = curs.fetchall()

1.Spring 2.Summer 3.Fall 4.Winter 1
1.Morning 2.Afternoon 3.Evening 4.Midnight 2
1.눈/비 2.맑음 3.흐림 3


In [32]:
# Transform to Pandas Dataframe
import pandas as pd
dataset_result = pd.DataFrame(dataset_result)
grid_result = pd.DataFrame(grid_result)
result = pd.concat([dataset_result, grid_result])
result['crime_num'] = result.groupby(['grid']).grid.transform('count') -1
result = result.drop_duplicates()
result = result.sort_values('grid')
result

,grid,CCTV_num,police_value,resident,floating,crime_num
0,275,0,2,105,118,4
1,191,0,1,110,126,3
2,690,0,1,118,124,4
3,786,0,0,111,118,2
4,287,0,0,110,126,4
...,...,...,...,...,...,...
920,1025,1,0,133,119,1
921,911,0,1,150,129,1
922,996,0,0,109,110,1
924,949,3,0,123,124,1
